In [21]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster, window_averaged_shuffle
from ieeg.viz.mri import gen_labels

# from utils import make_or_load_subjects_electrodes_to_ROIs_dict, load_acc_arrays, calculate_RTs, save_channels_to_file, save_sig_chans, \
#       load_sig_chans, channel_names_to_indices, filter_and_average_epochs, permutation_test, perform_permutation_test_across_electrodes, perform_permutation_test_within_electrodes, \
#       add_accuracy_to_epochs, load_mne_objects, create_subjects_mne_objects_dict, extract_significant_effects, convert_dataframe_to_serializable_format, \
#       perform_modular_anova, make_plotting_parameters, plot_significance

import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
# still need to test if the permutation test functions load in properly.
import pandas as pd
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# from src.analysis.power.roi_analysis import blah_blah
from src.analysis.config import experiment_conditions, plotting_parameters
import src.analysis.utils.general_utils as utils # import utils functions one by one by name

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

choose which subjects you wanna run (has to be a list, even if just one subject)

In [3]:
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']
subjects = ['D0057']

### make or load subjects electrodes to rois dict

In [4]:
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = r'C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config'
subjects_electrodestoROIs_dict = utils.make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                                filename='subjects_electrodestoROIs_dict.json')

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Failed to decode JSON from C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json: Expecting value: line 1 column 1 (char 0)
No dictionary found. Looks like it's our lucky day to create one!
D0057
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\de

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


outlier round 1 channels: ['RAMT8']
outlier round 2 channels: ['RAMT8', 'RPI16']
outlier round 2 channels: ['RAMT8', 'RPI16', 'LAMT12']
Reading 0 ... 6198083  =      0.000 ...  3026.408 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
DEBUG: For subject 'D57', attempting to read CSV: 'C:\Users\jz421\Box\ECoG_Recon\D57\elec_recon\D57_elec_location_radius_10mm_aparc.a2009s+aseg.mgz.csv'
Saved subjects_dict to C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json
Loaded data from C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json
Dictionary created and loaded successfully. Let's roll!


check number of elecs

In [5]:
# Calculate the total number of electrodes in the 'default_dict' across subjects
total_electrodes = utils.count_electrodes_across_subjects(subjects_electrodestoROIs_dict, subjects)
print('total elecs:', total_electrodes)

total elecs: 175


load behavioral data (combinedData.csv) and map blockType to congruency and switch proportions in the behavioral data

In [6]:
combined_data = pd.read_csv(r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\combinedData.csv')
combined_data[['congruencyProportion', 'switchProportion']] = combined_data.apply(utils.map_block_type, axis=1)

### load epochs objects for chosen conditions for all subjects in a dictionary

In [7]:
task='GlobalLocal'
conditions = experiment_conditions.stimulus_switch_type_conditions # set this to whichever conditions you're running
stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10"
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_0.0-30.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_100sec_within1-101sec_experimentStartBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_stat_func_ttest_ind_equal_var_False"

elif response_locked:
    # epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind"
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"

condition_names = [condition for condition in conditions.keys()]

In [8]:
if conditions == experiment_conditions.stimulus_conditions:
    conditions_save_name = 'stimulus_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_experiment_conditions:
    conditions_save_name = 'stimulus_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_main_effect_conditions:
    conditions_save_name = 'stimulus_main_effect_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwpc_conditions:
    conditions_save_name = 'stimulus_lwpc_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwps_conditions:
    conditions_save_name = 'stimulus_lwps_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_big_letter_conditions:
    conditions_save_name = 'stimulus_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_small_letter_conditions:
    conditions_save_name = 'stimulus_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_task_conditions:
    conditions_save_name = 'stimulus_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_congruency_conditions:
    conditions_save_name = 'stimulus_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_switch_type_conditions:
    conditions_save_name = 'stimulus_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

elif conditions == experiment_conditions.response_conditions:
    conditions_save_name = 'response_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_experiment_conditions:
    conditions_save_name = 'response_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_big_letter_conditions:
    conditions_save_name = 'response_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_small_letter_conditions:
    conditions_save_name = 'response_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_task_conditions:
    conditions_save_name = 'response_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_congruency_conditions:
    conditions_save_name = 'response_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_switch_type_conditions:
    conditions_save_name = 'response_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
    
# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
subjects_mne_objects = utils.create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True, error_trials_only=False)

Loading data for subject: D0057
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1_power_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline corr

c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:403: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs)


    Original shape: (212, 175, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 111.9
    Max NaN count in a trial: 1282
    Stimulus_r: 212 valid trials out of 212
  Loading condition: Stimulus_s with parameters: {'BIDS_events': ['Stimulus/s25.0', 'Stimulus/s75.0'], 'switchType': 's'}
Not setting metadata
189 matching events found
No baseline correction applied


c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:403: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs)


    Original shape: (189, 175, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 135.7
    Max NaN count in a trial: 1923
    Stimulus_s: 189 valid trials out of 189
  Loading condition: Stimulus_r with parameters: {'BIDS_events': ['Stimulus/r25.0', 'Stimulus/r75.0'], 'switchType': 'r'}
Not setting metadata
212 matching events found
No baseline correction applied


c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:403: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs)


    Original shape: (212, 175, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 111.9
    Max NaN count in a trial: 1282
    Stimulus_r: 212 valid trials out of 212
  Loading condition: Stimulus_s with parameters: {'BIDS_events': ['Stimulus/s25.0', 'Stimulus/s75.0'], 'switchType': 's'}
Not setting metadata
189 matching events found
No baseline correction applied


c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:403: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs)


    Original shape: (189, 175, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 135.7
    Max NaN count in a trial: 1923
    Stimulus_s: 189 valid trials out of 189


### load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [9]:
sig_chans_per_subject = utils.get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0057


### get the significant electrodes across subjects for each ROI of interest

dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup
ACC based on Destrieux et al 2010 definition is G_and_S_cingul-Ant

In [10]:
# rois_dict = {
#     # 'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     # 'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
#     # 'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
#     'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
# }

# the cns 24/sfn 24 poster plots need just one roi. Fix all this code later. 10/1.
# rois_dict = {
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"]
# }

rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
}

rois = list(rois_dict.keys())
electrodes_per_subject_roi, sig_electrodes_per_subject_roi = utils.make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
Subject D0057 significant G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes: ['RAI6', 'RAMF14']
For subject D0057, G_cuneus, G_and_S_occipital_inf, G_occipital_middle, G_occipital_sup, G_oc-temp_lat-fusifor, G_oc-temp_med-Lingual, Pole_occipital, S_calcarine, S_oc_middle_and_Lunatus, S_oc_sup_and_transversal, S_occipital_ant electrodes are: ['RPIT1']
Subject D0057 significant G_cuneus, G

In [11]:
print('total sig elecs:', sum(len(sig_chans_per_subject[sub]) for sub in sig_chans_per_subject))

total sig elecs: 74


get number of sig and all electrodes per subject and across subjects

In [16]:
for roi in rois:
    for sub in subjects:
        sig_elecs = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        all_elecs = electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        print(f"Subject {sub}, ROI {roi}, Num of Sig Electrodes: {len(sig_elecs)}, Num of All Electrodes: {len(all_elecs)}")

Subject D0057, ROI lpfc, Num of Sig Electrodes: 2, Num of All Electrodes: 17
Subject D0057, ROI occ, Num of Sig Electrodes: 1, Num of All Electrodes: 1


In [12]:
# Example usage:
total_electrodes_info = utils.calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])

Total number of significant lpfc electrodes across all subjects: 2
Total number of lpfc electrodes across all subjects: 17
Total number of significant occ electrodes across all subjects: 1
Total number of occ electrodes across all subjects: 1


check sampling rates to make sure they're all the same

In [13]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = utils.check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)

Subject D0057 has the expected sampling rate: 256.0 Hz.


TODO: stats next (maybe do time perm cluster with t-test as follow-up test for ANOVA significance, let's try using MNE functions for ANOVA and perm test)   
TODO: add in code for selecting electrodes of a specific ROI  
TODO: plotting average traces

In [23]:
plotting_parameters.plotting_parameters['Stimulus_r']

{'condition_parameter': 'repeat', 'color': 'blue', 'line_style': '-'}

In [ ]:
# TODO: go through all this code to verify it, it's made from Claude 6/29/25.

def create_roi_grand_average(subjects_mne_objects, subjects, roi, sig_electrodes_per_subject_roi, electrodes_per_subject_roi 
                           condition_names, mne_object_type='HG_ev1_power_rescaled'):
    """
    Create grand average evoked objects for each condition across all electrodes in an ROI.
    
    Parameters:
    -----------
    subjects_mne_objects : dict
        Nested dictionary with MNE objects
    subjects : list
        List of subject IDs
    roi : str
        ROI name (e.g., 'lpfc', 'occ')
    sig_electrodes_per_subject_roi : dict
        Dictionary mapping ROIs to subjects and their significant electrodes
    electrodes_per_subject_roi : dict
        Dictionary mapping ROIs to subjects and their electrodes
    condition_names : list
        List of condition names to process
    mne_object_type : str
        Which MNE object to use (default: 'HG_ev1_power_rescaled')
    
    Returns:
    --------
    dict
        Dictionary with condition names as keys and grand average evoked objects as values
    """
    grand_averages_sig_electrodes = {}
    grand_averages_all_electrodes = {}
    
    for condition_name in condition_names:
        all_evokeds_sig_electrodes = []
        all_evokeds_all_electrodes = []
        
        for sub in subjects:
            sig_electrodes = sig_electrodes_per_subject_roi[roi].get(sub, [])
            all_electrodes = electrodes_per_subject_roi[roi].get(sub, [])
            
            if not sig_electrodes:
                continue
                
            # Get the epochs for this subject and condition
            epochs = subjects_mne_objects[sub][condition_name][mne_object_type].copy()
            
            # Pick only significant electrodes
            epochs_sig_electrodes = epochs.pick_channels(sig_electrodes)
            epochs_all_electrodes = epochs.pick_channels(all_electrodes)
            
            # Create evoked for this subject (trial average)
            evoked_sig_electrodes = epochs_sig_electrodes.average()
            evoked_all_electrodes = epochs_all_electrodes.average()
            
            # Store each electrode's data separately for grand averaging
            for ch_idx, ch_name in enumerate(evoked_sig_electrodes.ch_names):
                # Create a copy with just this channel
                evoked_single_electrode = evoked_sig_electrodes.copy().pick_channels([ch_name])
                all_evokeds_sig_electrodes.append(evoked_single_electrode)

            # Store each electrode's data separately for grand averaging
            for ch_idx, ch_name in enumerate(evoked_all_electrodes.ch_names):
                # Create a copy with just this channel
                evoked_single_electrode = evoked_all_electrodes.copy().pick_channels([ch_name])
                all_evokeds_all_electrodes.append(evoked_single_electrode)

        if all_evokeds_sig_electrodes:
            # Create grand average across all electrodes from all subjects
            grand_avg = mne.grand_average(all_evokeds_sig_electrodes)
            grand_averages_sig_electrodes[condition_name] = grand_avg
            
        if all_evokeds_all_electrodes:
            # Create grand average across all electrodes from all subjects
            grand_avg = mne.grand_average(all_evokeds_all_electrodes)
            grand_averages_all_electrodes[condition_name] = grand_avg
            
    return grand_averages_sig_electrodes, grand_averages_all_electrodes

def plot_roi_comparison(grand_averages, condition_names, plotting_parameters, roi_name, 
                       title=None, ylim=None, show_sensors=False, show_legend=True,
                       time_unit='s', ci='sem', picks=None, axes=None, show=True):
    """
    Plot comparison of conditions for an ROI using MNE's plot_compare_evokeds.
    
    Parameters:
    -----------
    grand_averages : dict
        Dictionary with condition names as keys and evoked objects as values
    condition_names : list
        List of condition names to plot
    plotting_parameters : dict
        Dictionary with plotting parameters for each condition
    roi_name : str
        Name of the ROI for the title
    title : str, optional
        Custom title for the plot
    ylim : dict or tuple, optional
        Y-axis limits
    show_sensors : bool
        Whether to show sensor locations
    show_legend : bool or str
        Whether to show legend
    time_unit : str
        Time unit for x-axis
    ci : float or 'sem' or callable
        Confidence interval to plot
    picks : str or list
        Channels to pick
    axes : matplotlib axes
        Axes to plot on
    show : bool
        Whether to show the plot
    
    Returns:
    --------
    fig : matplotlib figure
        The figure object
    """
    # Prepare evokeds dictionary with custom styling
    evokeds_dict = {}
    
    for condition_name in condition_names:
        if condition_name in grand_averages:
            # Find the plotting parameter key that matches this condition
            param_key = None
            for key in plotting_parameters.keys():
                if condition_name in key:
                    param_key = key
                    break
            
            if param_key and param_key in plotting_parameters:
                params = plotting_parameters[param_key]
                # Create custom label with parameters
                label = params.get('condition_parameter', condition_name)
                evokeds_dict[label] = grand_averages[condition_name]
            else:
                evokeds_dict[condition_name] = grand_averages[condition_name]
    
    # Set up colors and linestyles
    colors = []
    linestyles = []
    
    for condition_name in condition_names:
        param_key = None
        for key in plotting_parameters.keys():
            if condition_name in key:
                param_key = key
                break
        
        if param_key and param_key in plotting_parameters:
            params = plotting_parameters[param_key]
            colors.append(params.get('color', 'black'))
            linestyles.append(params.get('line_style', '-'))
        else:
            colors.append('black')
            linestyles.append('-')
    
    # Create the plot
    if title is None:
        title = f'{roi_name.upper()} - Condition Comparison'
    
    fig = mne.viz.plot_compare_evokeds(
        evokeds_dict,
        colors=colors,
        linestyles=linestyles,
        picks=picks,
        axes=axes,
        show_sensors=show_sensors,
        show_legend=show_legend,
        split_legend=False,
        ci=ci,
        title=title,
        ylim=ylim,
        time_unit=time_unit,
        show=show
    )
    
    return fig

def plot_roi_conditions_separate(subjects_mne_objects, subjects, roi, sig_electrodes_per_subject_roi,
                               condition_names, plotting_parameters, mne_object_type='HG_ev1_power_rescaled',
                               ncols=2, figsize=None, ylim=None, show=True):
    """
    Plot each condition separately in subplots for an ROI.
    
    Parameters:
    -----------
    subjects_mne_objects : dict
        Nested dictionary with MNE objects
    subjects : list
        List of subject IDs
    roi : str
        ROI name
    sig_electrodes_per_subject_roi : dict
        Dictionary mapping ROIs to subjects and their significant electrodes
    condition_names : list
        List of condition names to plot
    plotting_parameters : dict
        Dictionary with plotting parameters
    mne_object_type : str
        Which MNE object to use
    ncols : int
        Number of columns for subplots
    figsize : tuple
        Figure size
    ylim : dict or tuple
        Y-axis limits
    show : bool
        Whether to show the plot
    
    Returns:
    --------
    fig : matplotlib figure
        The figure object
    """
    nrows = int(np.ceil(len(condition_names) / ncols))
    
    if figsize is None:
        figsize = (6 * ncols, 4 * nrows)
    
    fig, axes = plt.subplots(nrows, ncols, figsize=figsize, squeeze=False)
    axes = axes.flatten()
    
    # Create grand averages
    grand_averages = create_roi_grand_average(
        subjects_mne_objects, subjects, roi, sig_electrodes_per_subject_roi,
        condition_names, mne_object_type
    )
    
    for idx, condition_name in enumerate(condition_names):
        if idx >= len(axes):
            break
            
        ax = axes[idx]
        
        if condition_name in grand_averages:
            evoked = grand_averages[condition_name]
            
            # Find plotting parameters
            param_key = None
            for key in plotting_parameters.keys():
                if condition_name in key:
                    param_key = key
                    break
            
            if param_key and param_key in plotting_parameters:
                params = plotting_parameters[param_key]
                color = params.get('color', 'black')
                linestyle = params.get('line_style', '-')
                label = params.get('condition_parameter', condition_name)
            else:
                color = 'black'
                linestyle = '-'
                label = condition_name
            
            # Plot the evoked response
            evoked.plot(
                axes=ax,
                show=False,
                titles=f'{roi.upper()} - {label}',
                time_unit='s',
                ylim=ylim,
                spatial_colors=False,
                gfp=False,
                window_title=None
            )
            
            # Customize the line appearance
            lines = ax.get_lines()
            if lines:
                lines[0].set_color(color)
                lines[0].set_linestyle(linestyle)
                lines[0].set_linewidth(2)
            
            # Add shaded error region for SEM
            times = evoked.times
            data = evoked.data[0]  # Since we have only one channel after grand averaging
            
            # Calculate SEM across all individual electrode evokeds
            all_data = []
            for sub in subjects:
                sig_electrodes = sig_electrodes_per_subject_roi[roi].get(sub, [])
                if sig_electrodes:
                    epochs = subjects_mne_objects[sub][condition_name][mne_object_type].copy()
                    epochs_picked = epochs.pick_channels(sig_electrodes)
                    evoked_sub = epochs_picked.average()
                    all_data.extend(evoked_sub.data)
            
            if all_data:
                all_data = np.array(all_data)
                sem = np.std(all_data, axis=0, ddof=1) / np.sqrt(len(all_data))
                ax.fill_between(times, data - sem, data + sem, alpha=0.3, color=color)
    
    # Remove empty subplots
    for idx in range(len(condition_names), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.tight_layout()
    
    if show:
        plt.show()
    
    return fig

def plot_roi_comparison_custom(subjects_mne_objects, subjects, roi, sig_electrodes_per_subject_roi,
                             condition_names, plotting_parameters, mne_object_type='HG_ev1_power_rescaled',
                             title=None, figsize=(10, 6), ylim=None, show_sem=True, 
                             show_legend=True, show=True):
    """
    Create a custom plot comparing conditions with more control over appearance.
    
    Parameters:
    -----------
    subjects_mne_objects : dict
        Nested dictionary with MNE objects
    subjects : list
        List of subject IDs
    roi : str
        ROI name
    sig_electrodes_per_subject_roi : dict
        Dictionary mapping ROIs to subjects and their significant electrodes
    condition_names : list
        List of condition names to plot
    plotting_parameters : dict
        Dictionary with plotting parameters
    mne_object_type : str
        Which MNE object to use
    title : str
        Plot title
    figsize : tuple
        Figure size
    ylim : tuple
        Y-axis limits
    show_sem : bool
        Whether to show SEM shading
    show_legend : bool
        Whether to show legend
    show : bool
        Whether to show the plot
    
    Returns:
    --------
    fig, ax : matplotlib figure and axes
    """
    fig, ax = plt.subplots(figsize=figsize)
    
    # Create grand averages
    grand_averages = create_roi_grand_average(
        subjects_mne_objects, subjects, roi, sig_electrodes_per_subject_roi,
        condition_names, mne_object_type
    )
    
    for condition_name in condition_names:
        if condition_name not in grand_averages:
            continue
            
        evoked = grand_averages[condition_name]
        times = evoked.times
        data = evoked.data[0]  # Grand average data
        
        # Find plotting parameters
        param_key = None
        for key in plotting_parameters.keys():
            if condition_name in key:
                param_key = key
                break
        
        if param_key and param_key in plotting_parameters:
            params = plotting_parameters[param_key]
            color = params.get('color', 'black')
            linestyle = params.get('line_style', '-')
            label = params.get('condition_parameter', condition_name)
        else:
            color = 'black'
            linestyle = '-'
            label = condition_name
        
        # Plot the mean
        ax.plot(times, data, color=color, linestyle=linestyle, linewidth=2, label=label)
        
        if show_sem:
            # Calculate SEM across all individual electrode data
            all_data = []
            for sub in subjects:
                sig_electrodes = sig_electrodes_per_subject_roi[roi].get(sub, [])
                if sig_electrodes:
                    epochs = subjects_mne_objects[sub][condition_name][mne_object_type].copy()
                    epochs_picked = epochs.pick_channels(sig_electrodes)
                    evoked_sub = epochs_picked.average()
                    all_data.extend(evoked_sub.data)
            
            if all_data:
                all_data = np.array(all_data)
                sem = np.std(all_data, axis=0, ddof=1) / np.sqrt(len(all_data))
                ax.fill_between(times, data - sem, data + sem, alpha=0.2, color=color)
    
    # Customize plot
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Power')
    ax.axhline(y=0, color='black', linestyle='--', alpha=0.3)
    ax.axvline(x=0, color='black', linestyle='--', alpha=0.3)
    
    if title is None:
        title = f'{roi.upper()} - Condition Comparison'
    ax.set_title(title)
    
    if ylim:
        ax.set_ylim(ylim)
    
    if show_legend:
        ax.legend(loc='best')
    
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if show:
        plt.show()
    
    return fig, ax

# Example usage function
def plot_roi_analysis(subjects_mne_objects, subjects, rois_dict, sig_electrodes_per_subject_roi,
                     condition_names, plotting_parameters, mne_object_type='HG_ev1_power_rescaled'):
    """
    Plot analysis for multiple ROIs.
    
    Parameters:
    -----------
    subjects_mne_objects : dict
        Nested dictionary with MNE objects
    subjects : list
        List of subject IDs
    rois_dict : dict
        Dictionary mapping ROI names to brain regions
    sig_electrodes_per_subject_roi : dict
        Dictionary mapping ROIs to subjects and their significant electrodes
    condition_names : list
        List of condition names
    plotting_parameters : dict
        Dictionary with plotting parameters
    mne_object_type : str
        Which MNE object to use
    
    Returns:
    --------
    figures : dict
        Dictionary with ROI names as keys and figure objects as values
    """
    figures = {}
    
    for roi_name in rois_dict.keys():
        print(f"Plotting {roi_name}...")
        
        # Create grand averages for this ROI
        grand_averages = create_roi_grand_average(
            subjects_mne_objects, subjects, roi_name, sig_electrodes_per_subject_roi,
            condition_names, mne_object_type
        )
        
        # Method 1: Using MNE's plot_compare_evokeds
        fig1 = plot_roi_comparison(
            grand_averages, condition_names, plotting_parameters, roi_name,
            title=f'{roi_name.upper()} - MNE Comparison Plot',
            show=False
        )
        
        # Method 2: Custom plot with more control
        fig2, ax = plot_roi_comparison_custom(
            subjects_mne_objects, subjects, roi_name, sig_electrodes_per_subject_roi,
            condition_names, plotting_parameters, mne_object_type,
            title=f'{roi_name.upper()} - Custom Comparison Plot',
            show=False
        )
        
        # Method 3: Separate subplots for each condition
        fig3 = plot_roi_conditions_separate(
            subjects_mne_objects, subjects, roi_name, sig_electrodes_per_subject_roi,
            condition_names, plotting_parameters, mne_object_type,
            title=f'{roi_name.upper()} - Conditions Separately',
            show=False
        )
        
        figures[roi_name] = {'mne_plot': fig1, 'custom_plot': fig2, 'separate_plots': fig3}
    
    return figures

{'Stimulus_r': {'HG_ev1_power_rescaled': <EpochsArray | 212 events (all good), -1 – 1.5 s (baseline off), ~181.6 MB, data loaded,
   'Stimulus/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount227.0/BlockTrialCount3.0/ReactionTime883.0/Accuracy1.0/D57': 1
   'Stimulus/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount228.0/BlockTrialCount4.0/ReactionTime766.0/Accuracy1.0/D57': 1
   'Stimulus/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount241.0/BlockTrialCount17.0/ReactionTime499.0/Accuracy1.0/D57': 1
   'Stimulus/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount257.0/BlockTrialCount33.0/ReactionTime616.0/Accuracy1.0/D57': 1
   'Stimulus/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLet